# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [148]:
%matplotlib inline
# import numpy and pandas
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import f_oneway
from scipy.stats import linregress
import statsmodels.api as sm
from statsmodels.formula.api import ols
from scipy.stats import f
import re

# Challenge 1 - Analysis of Variance

In this part of the lesson, we will perform an analysis of variance to determine whether the factors in our model create a significant difference in the group means. We will be examining a dataset of FIFA players. We'll start by loading the data using the code in the cell below.

In [149]:
# Run this code:

fifa = pd.read_csv('fifa.csv')

Let's examine the dataset by looking at the `head`.

In [151]:
# Your code here:

def valuenumeric(x):
    money=x["Value"]
    money=re.sub(r'[€Mm]','',money)
    if "k" in str(money).lower():
        money=re.sub(r'[kK]','',money)
        money=float(money)/1000
    return {
        "ValueNumeric":float(money)
    }
fifa2=fifa.apply(valuenumeric,result_type="expand",axis=1)
fifa3=pd.concat([fifa,fifa2],axis=1)
fifa3.head()

,Name,Age,Nationality,Overall,Potential,Club,Value,Preferred Foot,Position,Weak Foot,...,SprintSpeed,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,ValueNumeric
0,L. Messi,31,Argentina,94,94,FC Barcelona,€110.5M,Left,RF,4.0,...,86.0,72.0,59.0,94.0,48.0,22.0,94.0,94.0,75.0,110.5
1,Cristiano Ronaldo,33,Portugal,94,94,Juventus,€77M,Right,ST,4.0,...,91.0,88.0,79.0,93.0,63.0,29.0,95.0,82.0,85.0,77.0
2,Neymar Jr,26,Brazil,92,93,Paris Saint-Germain,€118.5M,Right,LW,5.0,...,90.0,81.0,49.0,82.0,56.0,36.0,89.0,87.0,81.0,118.5
3,De Gea,27,Spain,91,93,Manchester United,€72M,Right,GK,3.0,...,58.0,43.0,64.0,12.0,38.0,30.0,12.0,68.0,40.0,72.0
4,K. De Bruyne,27,Belgium,91,92,Manchester City,€102M,Right,RCM,5.0,...,76.0,90.0,75.0,91.0,76.0,61.0,87.0,94.0,79.0,102.0


Player's values are expressed in millions of euros. We would like this column to be numeric. Therefore, let's create a numeric value column. Do this by stripping all non-numeric characters from each cell. Assign this new data to `ValueNumeric`. There is no need to multiply the value to be expressed in millions. 

In [152]:
# Your code here:
fifa3.Value.str.replace('M','000,000')

0        €110.5000,000
1           €77000,000
2        €118.5000,000
3           €72000,000
4          €102000,000
             ...      
18202             €60K
18203             €60K
18204             €60K
18205             €60K
18206             €60K
Name: Value, Length: 18207, dtype: object

In [157]:
fifa3['ValueNumeric'] = fifa3.Value.apply(lambda x: x[1:-1] if x[-1] != "K" else "."+ x[1:-1])
fifa3['Value'].unique
fifa3.ValueNumeric.str.replace('M','000,000')


0        110.5
1           77
2        118.5
3           72
4          102
         ...  
18202      .60
18203      .60
18204      .60
18205      .60
18206      .60
Name: ValueNumeric, Length: 18207, dtype: object

#### We'd like to determine whether a player's preffered foot and position have an impact on their value. 

Using the `statsmodels` library, we are able to produce an ANOVA table without munging our data. Create an ANOVA table with value as a function of position and preferred foot. Recall that pivoting is performed by the `C` function.

Hint: For columns that have a space in their name, it is best to refer to the column using the dataframe (For example: for column `A`, we will use `df['A']`).

In [161]:
# Your code here:
from statsmodels.formula.api import ols
import statsmodels.api as sm

formula = 'ValueNumeric ~ C(fifa3["Preferred Foot"])+C(Position)'

modelo = ols(formula=formula,data=fifa3).fit()
anova_table=sm.stats.anova_lm(modelo,type='II')
anova_table

ValueError: shape mismatch: value array of shape (2,218) could not be broadcast to indexing result of shape (2,2)

In [130]:
# No se por que me lo marca mal, si lo hice con el documento que subio el profesor

What is your conclusion from this ANOVA?

In [132]:
# Your conclusions here:
# No me salio la tabla ANOVA, pero la hice a mano y en base a los resultados que obtuve puedo concluir que:
# Se concluye que tanto el preffered foot y la position si son rechazados por la f, sin embargo cuando se hace la combinacion de ambos ya no se rechaza, y si una no se rechaza, se tiene que volver a hacer el analisis ya que la anova multiple se tienen que rechazar todas.

After looking at a model of both preffered foot and position, we decide to create an ANOVA table for nationality. Create an ANOVA table for numeric value as a function of nationality.

In [140]:
# Your code here:
formula = 'ValueNumeric ~ C(Nationality)'
modelo = ols(formula=formula, data=fifa).fit()
anova_table=sm.stats.anova_lm(modelo, typ ='II')
anova_table


ValueError: shapes (18207,217) and (18207,217) not aligned: 217 (dim 1) != 18207 (dim 0)

What is your conclusion from this ANOVA?

In [ ]:
# No se pudo ver la tabla ANOVA de nuevo, pero a mano me salio que se rehcaza Ho lo que significa que en un 95% de confianza se asume que el % de nacionalidad es optimo.

# Challenge 2 - Linear Regression

Our goal with using linear regression is to create a mathematical model that will enable us to predict the outcome of one variable using one or more additional independent variables.

We'll start by ensuring there are no missing values. Examine all variables for all missing values. If there are missing values in a row, remove the entire row.

In [162]:
# Your code here:
fifa4=fifa3.dropna()
print(fifa3.shape,fifa4.shape)

(18207, 21) (17918, 21)


Using the FIFA dataset, in the cell below, create a linear model predicting value using stamina and sprint speed. create the model using `statsmodels`. Print the model summary.

Hint: remember to add an intercept to the model using the `add_constant` function.

In [164]:
# Your code here:
Y=fifa4['ValueNumeric']
X=sm.add_constant(fifa4[['SprintSpeed','Stamina']])

modelo=sm.OLS(Y, X).fit()
predictions=modelo.predict(X)
modelo.summary()

ValueError: Pandas data cast to numpy dtype of object. Check input data with np.asarray(data).

Report your findings from the model summary. In particular, report about the model as a whole using the F-test and how much variation is predicted by the model using the r squared.

In [116]:
# Your conclusions here:



Next, create a second regression model predicting value using potential. Create the model using `statsmodels` and print the model summary. Remember to add a constant term.

In [12]:
# Your code here:
modelo.summary()

Report your findings from the model summary. In particular, report about the model as a whole using the F-test and how much variation is predicted by the model using the r squared.

In [13]:
# Your conclusions here:



Plot a scatter plot of value vs. potential. Do you see a linear relationship?

In [166]:
# Your code here:

fifa['ValueNumeric']=fifa4['ValueNumeric'].astype('float64')
fifa[Potencial]=fifa4['Potential'].astype('float64')

def linear_regresion(q,m):
    slope,intercept,r_value,std_err=linregress(q,m)
    
    def fn(x):
        return x * slope + intercept
    x = np.arange(q.min(),q.max(),1)
    
    y = [fn(x0) for x0 in x]
    plt.plot(x,y,'r')
    plt.scatter(q,m)
    
linear_regresion(fifa4['Potential'],fifa4['ValueNumeric'])

ValueError: could not convert string to float: 